## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-11-10-51-00 +0000,wsj,SoftBank Sells Its Nvidia Stake for $5.8 Billion,https://www.wsj.com/business/earnings/softbank...
1,2025-11-11-10-50-00 +0000,wsj,German Financial Sentiment Weakens on Worries ...,https://www.wsj.com/economy/central-banking/ge...
2,2025-11-11-10-45-28 +0000,nyt,A Syrian Village and the Long Road to the Whit...,https://www.nytimes.com/2025/11/10/world/middl...
3,2025-11-11-10-45-21 +0000,bbc,"Is the US government shutdown over, and why di...",https://www.bbc.com/news/articles/crrj1znp0pyo...
4,2025-11-11-10-44-00 +0000,wsj,Potential U.S. Government Reopening Could Prov...,https://www.wsj.com/finance/currencies/asian-c...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2358/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
143,trump,41
19,shutdown,33
18,government,23
44,new,19
384,senate,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
112,2025-11-11-01-13-00 +0000,wsj,Trump threatened to dock the pay of air-traffi...,https://www.wsj.com/politics/policy/trump-thre...,136
98,2025-11-11-02-00-00 +0000,wsj,Senate Minority Leader Chuck Schumer wasn’t on...,https://www.wsj.com/politics/policy/progressiv...,131
189,2025-11-10-21-11-40 +0000,nypost,Trump pledges to ‘abide’ by deal to end govern...,https://nypost.com/2025/11/10/us-news/trump-pl...,125
81,2025-11-11-03-12-00 +0000,wsj,The decision by eight members of the Senate De...,https://www.wsj.com/politics/policy/democrats-...,124
84,2025-11-11-02-55-00 +0000,wsj,The Republican-led Senate late Monday passed a...,https://www.wsj.com/politics/policy/senate-pas...,117


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
112,136,2025-11-11-01-13-00 +0000,wsj,Trump threatened to dock the pay of air-traffi...,https://www.wsj.com/politics/policy/trump-thre...
98,75,2025-11-11-02-00-00 +0000,wsj,Senate Minority Leader Chuck Schumer wasn’t on...,https://www.wsj.com/politics/policy/progressiv...
296,63,2025-11-10-14-58-51 +0000,nypost,Supreme Court rejects Kim Davis bid to overtur...,https://nypost.com/2025/11/10/us-news/supreme-...
295,55,2025-11-10-15-03-47 +0000,cbc,8 people dead in car explosion near Red Fort i...,https://www.cbc.ca/news/world/india-explosion-...
242,47,2025-11-10-17-49-00 +0000,wsj,The BBC reported that President Trump has thre...,https://www.wsj.com/politics/policy/bbc-chairm...
274,43,2025-11-10-16-35-00 +0000,wsj,Sen. Bernie Sanders and a group of Democratic ...,https://www.wsj.com/politics/policy/the-politi...
31,34,2025-11-11-09-24-46 +0000,nypost,Families sue Camp Mystic over deadly Texas flo...,https://nypost.com/2025/11/11/us-news/families...
84,33,2025-11-11-02-55-00 +0000,wsj,The Republican-led Senate late Monday passed a...,https://www.wsj.com/politics/policy/senate-pas...
279,32,2025-11-10-16-15-03 +0000,nypost,Saudi doctor who killed 6 after plowing car in...,https://nypost.com/2025/11/10/world-news/docto...
286,31,2025-11-10-15-58-48 +0000,nypost,NYC realtor fired over controversial Zohran Ma...,https://nypost.com/2025/11/10/business/nyc-rea...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
